In [1]:
import time
from sc3dynlib import Dynlib, build_snippet
from sc3.all import *

INFO:sc3.synth.server:setting global variable 's' to 'localhost'


In [2]:
s.boot()
time.sleep(8)
play()

INFO:sc3.synth.server:booting server 'localhost' on address 127.0.0.1:57110
INFO:SERVER.stdout:Faust: supercollider.cpp: sc_api_version = 3
INFO:SERVER.stdout:Faust: FaustJPverbRaw numControls=11
INFO:SERVER.stdout:Found 305 LADSPA plugins
INFO:SERVER.stdout:Faust: supercollider.cpp: sc_api_version = 3
INFO:SERVER.stdout:Faust: FaustGreyholeRaw numControls=7
INFO:SERVER.stdout:JackDriver: client name is 'SuperCollider'
INFO:SERVER.stdout:SC_AudioDriver: sample rate = 48000.000000, driver's block size = 1024
INFO:SERVER.stdout:JackDriver: connected  system:capture_1 to SuperCollider:in_1
INFO:SERVER.stdout:JackDriver: connected  system:capture_2 to SuperCollider:in_2
INFO:SERVER.stdout:JackDriver: connected  SuperCollider:out_1 to system:playback_1
INFO:SERVER.stdout:JackDriver: connected  SuperCollider:out_2 to system:playback_2
INFO:SERVER.stdout:SuperCollider 3 server ready.
INFO:SERVER.stdout:JackDriver: max output latency 42.7 ms
INFO:sc3.synth._serverstatus:'localhost': requested 

In [3]:
cpp = """
#include <math.h>
#include <iostream>
#define PI 3.141592653589793238462

#define N_STATE 1
#define N_INPUTS 0
#define N_OUTPUTS 2
#define N_PARAMETERS 1

#define BLOCKWISE 1

extern "C"
{
    double ph=0;

    void settings(int x[])
    {
        x[0] = N_STATE;
        x[1] = N_OUTPUTS;
        x[2] = N_INPUTS;
        x[3] = N_PARAMETERS;
        x[4] = BLOCKWISE;
    }

    void setup(double dt, int N)
    {
        std::cout << "DemoSin setup\\n" << std::endl;
        ph = 0;
    }

    void process_blockwise(double **state_in, double **state_out, double **in, double **out, double *param, double dt, int N)
    {
        float f = param[0];        
        float dph = 2*PI*f*dt;
        for (int i = 0; i < N; i++)
        {
            ph += dph;
            out[0][i] = sin(ph);
            out[1][i] = sin(ph);
        } 
    }
}
"""

In [4]:
!rm libDemoSin.so
build_snippet('DemoSin', cpp)
!ls

0
DemoSin.cpp  DemoSin.ipynb  DemoSin.o  DemoSin.py  libDemoSin.so


In [5]:
@synthdef
def demoSin(amp=1.0, freq=200):
    gen = Dynlib.ar('DemoSin',freq)*amp
    Out(0, gen)
time.sleep(0.5)

[7, 68, 101, 109, 111, 83, 105, 110, OutputProxy.new('control', Control.kr([1.0, 200]), 1)]


In [6]:
o = demoSin(amp=1, freq=220)
time.sleep(3)

INFO:SERVER.stdout:Dynlib v0.2
INFO:SERVER.stdout:Plugin name: DemoSin
INFO:SERVER.stdout:Lib name: ./libDemoSin.so
INFO:SERVER.stdout:DemoSin: DLOPEN: ok
INFO:SERVER.stdout:DemoSin: N_STATE=1	 N_OUTPUTS=2	 N_INPUTS=0	 N_PARAMETERS=1	 BLOCKWISE=1
INFO:SERVER.stdout:RTALLOCS
INFO:SERVER.stdout:DemoSin: PARAMS 220
INFO:SERVER.stdout:DemoSin: OUT 0
INFO:SERVER.stdout:DemoSin: OUT 0
INFO:SERVER.stdout:SETUP
INFO:SERVER.stdout:DemoSin setup
INFO:SERVER.stdout:
INFO:SERVER.stdout:CALL


In [7]:
o.set('amp',0.5)
o.set('freq',300)

In [8]:
o.free()

INFO:SERVER.stdout:DemoSin: All FreeDemoSin: ClosedDynlib v0.2
INFO:SERVER.stdout:Plugin name: DemoSin
INFO:SERVER.stdout:Lib name: ./libDemoSin.so
INFO:SERVER.stdout:DemoSin: DLOPEN: ok
INFO:SERVER.stdout:DemoSin: N_STATE=1	 N_OUTPUTS=2	 N_INPUTS=0	 N_PARAMETERS=1	 BLOCKWISE=1
INFO:SERVER.stdout:RTALLOCS
INFO:SERVER.stdout:DemoSin: PARAMS 220
INFO:SERVER.stdout:DemoSin: OUT 0
INFO:SERVER.stdout:DemoSin: OUT 0
INFO:SERVER.stdout:SETUP
INFO:SERVER.stdout:DemoSin setup
INFO:SERVER.stdout:
INFO:SERVER.stdout:CALL


In [10]:
# CmdPeriod.run()